In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import lightgbm as lgb

# Pré-processamento #

### Base e encoder manual ###

In [2]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df_encod_manual = pd.DataFrame.copy(df)

In [5]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [6]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [7]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [8]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [9]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [10]:
previsores_label = df.iloc[:, 0:11].values

In [11]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [12]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [13]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [14]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [15]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [16]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [17]:
x_train, x_test, y_train, y_test = train_test_split(
    previsores, alvo,
    test_size = 0.3, random_state = 0
)

# Previsores #
<span style="font-size: 13px;">
<li> <strong>previsores</strong> -> Atributos codificados manualmente sem escalonamento.</li></br>
<li> <strong>previsoresHot_esc</strong> -> Atributos codificados com LabelEncoder e OneHotEncoder e escalonados.</li></br>
<li> <strong>previsores_esc</strong> -> Atributos codificados manualmente e escalonados.</li></br>
<li> <strong>previsores_hot</strong> -> Atributos codificados com OneHotEncoder sem escalonamento.</li></br>
<li> <strong>previsores_label</strong> -> Atributos codificados com LabelEncoder e sem escalonamento. </li></br>
</span>

# LightGBM #

<span style="font-size: 13px;">
O LightGBM baseia-se no conceito de Gradient Boosting, que combina vários modelos de aprendizado de máquina fracos para criar um modelo mais robusto. Ele utiliza árvores de decisão como modelos base.

**Gradient Boosting:**

O Gradient Boosting é uma técnica de aprendizado de máquina que cria um modelo preditivo combinando várias iterações de modelos simples. Cada modelo é ajustado para corrigir os erros do modelo anterior.

**Árvores de Decisão:**

As árvores de decisão são estruturas de árvore que dividem os dados em subconjuntos com base nas características, buscando maximizar a precisão da previsão.

**Fórmulas**

**Função de Perda (Loss Function):**

A função de perda é usada para medir a discrepância entre as previsões do modelo e os valores reais. Exemplos incluem a função de perda de entropia cruzada para classificação e a função de perda de erro quadrático médio para regressão.

**Gradiente e Hessiana:**

O Gradient Boosting otimiza a função de perda ajustando os modelos subsequentes na direção do gradiente da função de perda. O Hessiano é usado para ajustar a taxa de aprendizado durante o treinamento.


**Hiperparâmetros:**

- `num_leaves`: O número máximo de folhas em cada árvore.
- `learning_rate`: Taxa de aprendizado para atualizar os modelos.
- `max_depth`: A profundidade máxima de cada árvore.
- `min_data_in_leaf`: O número mínimo de amostras em cada folha.

**Parâmetros:**

- `objective`: O tipo de problema a ser resolvido, como "regressão" ou "classificação".
</span>

## Configuração do LightGBM ##
<span style="font-size: 13px;">

**Hiperparâmetros** </br>

**Controle de ajuste** </br>
<li>num_leaves : define o número de folhas a serem formadas em uma árvore. Não tem uma relação direta entre num_leaves e max_depth e, portanto, os dois não devem estar vinculados um ao outro. </li>

<li>max_depth : especifica a profundidade máxima ou nível até o qual a árvore pode crescer.  </li>

**Controle de velocidade** </br>

<li>learning_rate: taxa de aprendizagem, determina o impacto de cada árvore no resultado final.</li>

<li>max_bin : O valor menor de max_bin reduz muito tempo de procesamento, pois agrupa os valores do recurso em caixas discretas, o que é computacionalmente mais barato.</li>

**Controle de precisão** </br>

<li>num_leaves : valor alto produz árvores mais profundas com maior precisão, mas leva ao overfitting.</li>

<li>max_bin : valores altos tem efeito semelhante ao causado pelo aumento do valor de num_leaves e também torna mais lento o procedimento de treinamento.</li>
</span>

**Criar um DataSet para treino**

In [18]:
dataset = lgb.Dataset(
    x_train, label = y_train
)

**Dict de parametros e treinamento**

In [19]:
param = {
    'num_leaves': 150,
    'objective': 'binary',
    'max_depth': 2,
    'learning_rate': 0.05,
    'max_bin': 200,
}

In [20]:
lgbm = lgb.train(
    param, dataset,
    num_boost_round = 150,
)

[LightGBM] [Info] Number of positive: 352, number of negative: 289
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 394
[LightGBM] [Info] Number of data points in the train set: 641, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549142 -> initscore=0.197204
[LightGBM] [Info] Start training from score 0.197204
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

**Previsoes**

In [21]:
previsoes_test = lgbm.predict(x_test)

*As previsões não retornam valores binários, por isso devemos manualmente realizar a binarização, no caso das previsões acima, quando o valor for menor que 5 iremos considerar 0, e quando for maior ou igual a 5 iremos considerar como 1*

In [22]:
for i in range(0, 276):
    if previsoes_test[i] >= .5:
        previsoes_test[i] = 1
    else:
        previsoes_test[i] = 0

**Métricas**

In [23]:
acuracia_test = accuracy_score(y_test, previsoes_test) * 100.0
print("Acurácia: %.2f%%" % acuracia_test)

Acurácia: 85.51%


In [24]:
print(confusion_matrix(y_test, previsoes_test))

[[101  20]
 [ 20 135]]


In [25]:
print(classification_report(y_test, previsoes_test))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       121
           1       0.87      0.87      0.87       155

    accuracy                           0.86       276
   macro avg       0.85      0.85      0.85       276
weighted avg       0.86      0.86      0.86       276



**Análise dos dados de treino(verificar overfitting)**

In [26]:
previsoes_train = lgbm.predict(x_train)

In [27]:
for i in range(0, 641):
    if previsoes_train[i] >= .5:
        previsoes_train[i] = 1
    else:
        previsoes_train[i] = 0

In [28]:
acuracia_train = accuracy_score(y_train, previsoes_train) * 100.0
print("Acurácia: %.2f%%" % acuracia_train)

Acurácia: 91.26%


In [29]:
print(confusion_matrix(y_train, previsoes_train))

[[252  37]
 [ 19 333]]


### Cross Validation ###

In [30]:
kfold = KFold(n_splits = 30, shuffle = True, random_state = 5)

In [32]:
modelo = lgb.LGBMClassifier(
    num_leaves = 150,
    objective = 'binary',
    max_depth = 2,
    learning_rate = 0.05,
    max_bin = 200
)
resultado = cross_val_score(modelo, previsores, alvo, cv = kfold)

[LightGBM] [Info] Number of positive: 489, number of negative: 397
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 436
[LightGBM] [Info] Number of data points in the train set: 886, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.551919 -> initscore=0.208426
[LightGBM] [Info] Start training from score 0.208426
[LightGBM] [Info] Number of positive: 490, number of negative: 396
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000218 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 439
[LightGBM] [Info] Number of data points in the train set: 886, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.553047 -> initscore=0.212991
[LightGBM] [Info] S

In [33]:
print("Acurácia média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia média: 85.82%
